In [1]:
%run common.ipynb

In [2]:
import csv
import urllib

In [3]:
import bs4 as bs

In [4]:
from multiprocessing import Pool


In [5]:
programcode_to_id = pd.read_csv(PROGRAMCODE_TO_ID_FILE)
program_codes = set(programcode_to_id.code)

In [6]:
pd.read_html('http://fgosvo.ru/fgosvo/142/141/16/28', )[0]

,0,1,2
0,Код,Наименование направления,Текст стандарта
1,01.03.01,Математика,Скачать
2,01.03.02,Прикладная математика и информатика,Скачать
3,01.03.03,Механика и математическое моделирование,Скачать
4,01.03.04,Прикладная математика,Скачать


In [7]:
PAT_PROGRAM_LINK = re.compile('^\d{6}.*$')


def extract_program_link(dn):
    if not PAT_PROGRAM_LINK.match(dn.text):
        return
    yield urllib.parse.urljoin('http://fgosvo.ru/', dn.find('a')['href'])

def pagelist():
    pages = bs.BeautifulSoup(requests.get('http://fgosvo.ru/fgosvo/142/141/16').text)
    pages = map(extract_program_link, pages.find_all(class_='dn'))
    pages = itertools.chain.from_iterable(pages)
    return list(pages)
    

program_pages = pagelist()
program_pages

['http://fgosvo.ru/fgosvo/142/141/16/28',
 'http://fgosvo.ru/fgosvo/142/141/16/29',
 'http://fgosvo.ru/fgosvo/142/141/16/3',
 'http://fgosvo.ru/fgosvo/142/141/16/4',
 'http://fgosvo.ru/fgosvo/142/141/16/5',
 'http://fgosvo.ru/fgosvo/142/141/16/6',
 'http://fgosvo.ru/fgosvo/142/141/16/7',
 'http://fgosvo.ru/fgosvo/142/141/16/8',
 'http://fgosvo.ru/fgosvo/142/141/16/9',
 'http://fgosvo.ru/fgosvo/142/141/16/10',
 'http://fgosvo.ru/fgosvo/142/141/16/11',
 'http://fgosvo.ru/fgosvo/142/141/16/12',
 'http://fgosvo.ru/fgosvo/142/141/16/13',
 'http://fgosvo.ru/fgosvo/142/141/16/14',
 'http://fgosvo.ru/fgosvo/142/141/16/15',
 'http://fgosvo.ru/fgosvo/142/141/16/16',
 'http://fgosvo.ru/fgosvo/142/141/16/17',
 'http://fgosvo.ru/fgosvo/142/141/16/18',
 'http://fgosvo.ru/fgosvo/142/141/16/19',
 'http://fgosvo.ru/fgosvo/142/141/16/20',
 'http://fgosvo.ru/fgosvo/142/141/16/21',
 'http://fgosvo.ru/fgosvo/142/141/16/22',
 'http://fgosvo.ru/fgosvo/142/141/16/23',
 'http://fgosvo.ru/fgosvo/142/141/16/24',

In [8]:
PAT_PROGRAMCODE = re.compile(r'\d\d\.\d\d\.\d\d')
assert PAT_PROGRAMCODE.match('01.03.02')
assert not PAT_PROGRAMCODE.match('01,03,02')

In [9]:
def tr_to_record(tr):
    tds = tr.find_all('td')
    if len(tds) != 3:
        return
    program_code = tds[0].text.strip()
    program_code = PAT_PROGRAMCODE.search(program_code)
    if not program_code:
        return
    program_code = program_code.group(0)
    url = tds[2].find('a')['href']
    if not url.endswith('.pdf'):
        return
    url = urllib.parse.urljoin('http://fgosvo.ru/', url)
    yield (program_code, url)
    
def program_url_to_pdflist(url):
    pdfs = requests.get(url).text
    pdfs = bs.BeautifulSoup(pdfs)
    pdfs = pdfs.find('table').find_all('tr')
    pdfs = map(tr_to_record, pdfs)
    return itertools.chain.from_iterable(pdfs)
    

In [10]:
soup010302 = bs.BeautifulSoup(requests.get('http://fgosvo.ru/fgosvo/142/141/16/28').text)

In [11]:
list(itertools.chain.from_iterable(map(tr_to_record, soup010302.find('table').find_all('tr'))))

[('01.03.01',
  'http://fgosvo.ru/fgosvo/downloads/1239/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010301_B_3plus_12122017.pdf'),
 ('01.03.02',
  'http://fgosvo.ru/fgosvo/downloads/1240/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010302_B_3plus_12122017.pdf'),
 ('01.03.03',
  'http://fgosvo.ru/fgosvo/downloads/1241/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010303_B_3plus_12122017.pdf'),
 ('01.03.04',
  'http://fgosvo.ru/fgosvo/downloads/1242/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010304_B_3plus_12122017.pdf')]

In [12]:
list(program_url_to_pdflist('http://fgosvo.ru/fgosvo/142/141/16/3'))

[('03.03.01',
  'http://fgosvo.ru/fgosvo/downloads/1517/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F030301_B_3plus_22112017.pdf'),
 ('03.03.02',
  'http://fgosvo.ru/fgosvo/downloads/1518/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F030302_B_3plus_22112017.pdf'),
 ('03.03.03',
  'http://fgosvo.ru/fgosvo/downloads/1519/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F030303_B_3plus_22112017.pdf')]

* * *

In [13]:
with open('program_pdf_urls.txt', 'w') as f, Pool(5) as pool:
    pdflist = pool.imap(program_url_to_pdflist, program_pages)
    pdflist = itertools.chain.from_iterable(pdflist)
    pdflist = tqdm.tqdm(pdflist)
    
    out = csv.writer(f)
    out.writerow(['code', 'url'])
    i = 0
    def tee(pdf):
        out.writerow(pdf)
        return pdf
    pdflist = map(tee, pdflist)
    pdflist = list(pdflist)

176it [02:10,  1.35it/s]


In [14]:
len(pdflist)

176

In [15]:
len(program_codes)

459

In [16]:
! wc -l programs.csv

291 programs.csv


In [17]:
pdflist

[('01.03.01',
  'http://fgosvo.ru/fgosvo/downloads/1239/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010301_B_3plus_12122017.pdf'),
 ('01.03.02',
  'http://fgosvo.ru/fgosvo/downloads/1240/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010302_B_3plus_12122017.pdf'),
 ('01.03.03',
  'http://fgosvo.ru/fgosvo/downloads/1241/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010303_B_3plus_12122017.pdf'),
 ('01.03.04',
  'http://fgosvo.ru/fgosvo/downloads/1242/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010304_B_3plus_12122017.pdf'),
 ('02.03.01',
  'http://fgosvo.ru/fgosvo/downloads/991/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F020301_B_3plus_21062017.pdf'),
 ('02.03.02',
  'http://fgosvo.ru/fgosvo/downloads/992/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F020302_B_3plus_21062017.pdf'),
 ('02.03.03',
  'http://fgosvo.ru/fgosvo/downloads/993/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F020303_B_3plus_21062017.pdf'),
 ('03.03.01',
  'http:/

**Long story short**, some of the educational programs that I've parsed from minobrnauki reports are simply undocumented...

In [18]:
list(program_codes.difference([code for code, url in pdflist]))[:5]

['11.04.04', '12.05.01', '04.04.02', '41.04.05', '24.04.01']

In [19]:
%%time

def dl_pdf(code_and_url):
    code, url = code_and_url
    dst_path = os.path.join('programs', code + '.pdf')
    if not os.path.exists('programs'):
        os.mkdir('programs')
    with open(dst_path, 'wb') as out, requests.get(url, stream=True) as rsp:
        out.write(b'')
        for chunk in rsp.iter_content(1024):
            out.write(chunk)
    return dst_path

with Pool(5) as pool:
    downloaded = pool.imap(dl_pdf, pdflist)
    for d in tqdm.tqdm(downloaded):
        print('Finished downloading {}'.format(d))

1it [00:04,  4.16s/it]

Finished downloading programs/01.03.01.pdf
Finished downloading programs/01.03.02.pdf
Finished downloading programs/01.03.03.pdf
Finished downloading programs/01.03.04.pdf


5it [00:05,  3.04s/it]

Finished downloading programs/02.03.01.pdf


6it [00:06,  2.20s/it]

...

Finished downloading programs/09.03.03.pdf
Finished downloading programs/09.03.04.pdf
Finished downloading programs/10.03.01.pdf
Finished downloading programs/11.03.01.pdf


32it [02:29, 10.71s/it]

Finished downloading programs/11.03.02.pdf
Finished downloading programs/11.03.03.pdf
Finished downloading programs/11.03.04.pdf
Finished downloading programs/12.03.01.pdf
Finished downloading programs/12.03.02.pdf
Finished downloading programs/12.03.03.pdf
Finished downloading programs/12.03.04.pdf
Finished downloading programs/12.03.05.pdf
Finished downloading programs/13.03.01.pdf
Finished downloading programs/13.03.02.pdf
Finished downloading programs/13.03.03.pdf
Finished downloading programs/14.03.01.pdf
Finished downloading programs/14.03.02.pdf
Finished downloading programs/15.03.01.pdf
Finished downloading programs/15.03.02.pdf
Finished downloading programs/15.03.03.pdf
Finished downloading programs/15.03.04.pdf
Finished downloading programs/15.03.05.pdf
Finished downloading programs/15.03.06.pdf
Finished downloading programs/16.03.01.pdf
Finished downloading programs/16.03.02.pdf
Finished downloading programs/16.03.03.pdf
Finished downloading programs/18.03.01.pdf
Finished do

176it [03:47,  1.29s/it]

Finished downloading programs/45.03.03.pdf
Finished downloading programs/45.03.04.pdf
Finished downloading programs/46.03.04.pdf
Finished downloading programs/47.03.01.pdf
Finished downloading programs/47.03.02.pdf
Finished downloading programs/47.03.03.pdf
Finished downloading programs/49.03.01.pdf
Finished downloading programs/49.03.02.pdf
Finished downloading programs/49.03.03.pdf
Finished downloading programs/50.03.01.pdf
Finished downloading programs/50.03.02.pdf
Finished downloading programs/50.03.03.pdf
Finished downloading programs/50.03.04.pdf
Finished downloading programs/51.03.01.pdf
Finished downloading programs/51.03.02.pdf
Finished downloading programs/51.03.03.pdf
Finished downloading programs/51.03.04.pdf
Finished downloading programs/51.03.05.pdf
Finished downloading programs/51.03.06.pdf
Finished downloading programs/52.03.01.pdf
Finished downloading programs/52.03.02.pdf
Finished downloading programs/52.03.03.pdf
Finished downloading programs/52.03.04.pdf
Finished do

Not going to validate, will assume it's all ok

* * *

There are some problems with running `ghostscript` in conda/jupyter under firejail and `camelot-py` (which I'm using to extract tables) heavily depends on it, so there's `extract_fgosvo.py` in the project root which can be run outside conda enviroment. It's got a number of rather shameful hacks, takes a while to run, and needs increased `ulimit -n`.

 * * *
 
 Now, before we can get to OKPDTR... we need to map `profstandard_code`s into `OKZ`, via http://fgosvo.ru/docs/101/69/2
 
 
 And this time I feel that I don't want to waste time waiting for `camelot` to stop crashing...

In [42]:
PAT_CAT = re.compile(r'\b\d\d\b')


def dn_profstandard_cats(div):
    cat = PAT_CAT.search(div.text)
    if not cat:
        return
    cat = cat.group(0)
    a = div.find('a')
    name = a.text.strip()
    href = a['href']
    yield (cat, name, href)
    
def tee(writer):
    def _tee(x):
        writer(x)
        return x
    _tee.__name__ = 'tee'
    return _tee

def consume(iterator):
    collections.deque(iterator, maxlen=0)


with requests.get('http://fgosvo.ru/docs/101/69/2') as r, open('profstandard_cats.csv', 'w') as f:
    out = csv.writer(f)
    out.writerow(['cat', 'name', 'href'])
    ps_cats_page = bs.BeautifulSoup(r.text)
    ps_cats = ps_cats_page.find_all(class_='dn')
    ps_cats = map(dn_profstandard_cats, ps_cats)
    ps_cats = itertools.chain.from_iterable(ps_cats)
    ps_cats = map(tee(out.writerow), ps_cats)
    ps_cats = list(ps_cats)

In [43]:
ps_cats

[('01', 'ОБРАЗОВАНИЕ', '/docs/101/69/2/1'),
 ('02', 'ЗДРАВООХРАНЕНИЕ', '/docs/101/69/2/2'),
 ('03', 'СОЦИАЛЬНОЕ ОБСЛУЖИВАНИЕ', '/docs/101/69/2/3'),
 ('04', 'КУЛЬТУРА, ИСКУССТВО', '/docs/101/69/2/4'),
 ('05', 'ФИЗИЧЕСКАЯ КУЛЬТУРА И СПОРТ', '/docs/101/69/2/5'),
 ('06',
  'СВЯЗЬ, ИНФОРМАЦИОННЫЕ И КОММУНИКАЦИОННЫЕ ТЕХНОЛОГИИ',
  '/docs/101/69/2/6'),
 ('07',
  'АДМИНИСТРАТИВНО-УПРАВЛЕНЧЕСКАЯ И ОФИСНАЯ ДЕЯТЕЛЬНОСТЬ',
  '/docs/101/69/2/7'),
 ('08', 'ФИНАНСЫ И ЭКОНОМИКА', '/docs/101/69/2/8'),
 ('09', 'ЮРИСПРУДЕНЦИЯ', '/docs/101/69/2/9'),
 ('10',
  'АРХИТЕКТУРА, ПРОЕКТИРОВАНИЕ, ГЕОДЕЗИЯ, ТОПОГРАФИЯ И ДИЗАЙН',
  '/docs/101/69/2/10'),
 ('11',
  'СРЕДСТВА МАССОВОЙ ИНФОРМАЦИИ, ИЗДАТЕЛЬСТВО И ПОЛИГРАФИЯ',
  '/docs/101/69/2/11'),
 ('12', 'ОБЕСПЕЧЕНИЕ БЕЗОПАСНОСТИ', '/docs/101/69/2/12'),
 ('13', 'СЕЛЬСКОЕ ХОЗЯЙСТВО', '/docs/101/69/2/13'),
 ('14', 'ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА', '/docs/101/69/2/14'),
 ('15', 'РЫБОВОДСТВО И РЫБОЛОВСТВО', '/docs/101/69/2/15'),
 ('16', 'СТРОИТЕЛЬСТВО И ЖИЛИЩНО-КОММУНАЛЬНОЕ ХОЗ

In [44]:
ps_cats = [cat[-1] for cat in ps_cats]

In [49]:
ps_cats = [urllib.parse.urljoin('http://fgosvo.ru', url) for url in ps_cats]

In [60]:
PAT_PROFSTANDARD_CODE = re.compile(r'\b\d\d\.\d\d\d\b')


def parse_ps_tr(tr):
    tds = tr.find_all('td')
    if len(tds) != 3:
        raise Exception('expected 3 TDs')
        return
    ps = PAT_PROFSTANDARD_CODE.search(tds[0].text)
    if not ps:
        return
    ps = ps.group(0)
    name = tds[1].text.strip()
    a = tds[2].find('a')
    if not a:
        raise Exception('no href')
        return
    a =  a['href']
    yield ps, name, a

def caturl_to_psurls(caturl):
    with requests.get(caturl) as r:
        page = bs.BeautifulSoup(r.text)
    if not page:
        return []
    urls = map(parse_ps_tr, page.find(class_='st').find_all('tr'))
    urls = itertools.chain.from_iterable(urls)
    return list(urls)

with open('profstandard_pdf_urls.csv', 'w') as f, Pool(2) as pool:
    out = csv.writer(f)
    out.writerow(['ps', 'name', 'url'])
    urls = pool.imap(caturl_to_psurls, ps_cats)
    # urls = map(caturl_to_psurls, ps_cats)
    urls = tqdm.tqdm(urls)
    urls = itertools.chain.from_iterable(urls)
    urls = map(tee(out.writerow), urls)
    urls = list(urls)









0it [00:00, ?it/s]







1it [00:01,  1.73s/it]







3it [00:03,  1.46s/it]







5it [00:04,  1.19s/it]







6it [00:04,  1.04it/s]







7it [00:05,  1.05it/s]







8it [00:06,  1.17it/s]







9it [00:07,  1.15it/s]







10it [00:08,  1.19it/s]







11it [00:08,  1.23it/s]







12it [00:09,  1.23it/s]







13it [00:10,  1.19it/s]







14it [00:11,  1.38it/s]







15it [00:12,  1.18it/s]







16it [00:13,  1.18it/s]







17it [00:13,  1.27it/s]







18it [00:14,  1.20it/s]







19it [00:15,  1.33it/s]







20it [00:15,  1.35it/s]







21it [00:16,  1.37it/s]







22it [00:17,  1.26it/s]







23it [00:18,  1.42it/s]







24it [00:19,  1.07it/s]







25it [00:19,  1.43it/s]







26it [00:20,  1.28it/s]







27it [00:20,  1.66it/s]







28it [00:22,  1.20it/s]







30it [00:24,  1.08it/s]







31it [00:24,  1.31it/s]







32it [00:25,  1.19it/s]







33it [00:26,  1.28it/s]







34it [00:27,  1.03it/s]

In [61]:
urls

[('01.001',
  'Педагог (педагогическая деятельность в сфере дошкольного, начального общего, основного общего, среднего общего образования) (воспитатель, учитель)',
  '/docs/downloads/2/?f=%2Fuploadfiles%2Fprofstandart%2F01.001.pdf'),
 ('01.002',
  'Педагог-психолог (психолог в сфере образования)',
  '/docs/downloads/494/?f=%2Fuploadfiles%2Fprofstandart%2F01.002.pdf'),
 ('01.003',
  'Педагог дополнительного образования детей и взрослых',
  '/docs/downloads/508/?f=%2Fuploadfiles%2Fprofstandart%2F01.003.pdf'),
 ('01.004',
  'Педагог профессионального обучения, профессионального образования и дополнительного профессионального образования',
  '/docs/downloads/507/?f=%2Fuploadfiles%2Fprofstandart%2F01.004.pdf'),
 ('01.005',
  'Специалист в области воспитания',
  '/docs/downloads/835/?f=%2Fuploadfiles%2Fprofstandart%2F01.005.pdf'),
 ('01.006',
  'Мастер производственного обучения вождению транспортных средств соответствующих категорий и подкатегорий',
  '/docs/downloads/1200/?f=%2Fuploadfiles

In [66]:
%%time

def dl_pdf(code_and_url):
    code, url = code_and_url
    dst_path = os.path.join('ps', code + '.pdf')
    if not os.path.exists('ps'):
        os.mkdir('ps')
    with open(dst_path, 'wb') as out, requests.get(url, stream=True) as rsp:
        out.write(b'')
        for chunk in rsp.iter_content(1024):
            out.write(chunk)
    return dst_path

with Pool(8) as pool:
    downloaded = pool.imap(dl_pdf, ((code, urllib.parse.urljoin('http://fgosvo.ru', url))
                                    for (code, name, url) in urls))
    downloaded = tqdm.tqdm(downloaded)
    downloaded = list(downloaded)













0it [00:00, ?it/s]











1it [00:06,  6.43s/it]











2it [00:08,  5.20s/it]











3it [00:10,  4.22s/it]











4it [01:53, 33.81s/it]











46it [02:32, 23.95s/it]











50it [02:33, 16.78s/it]











54it [02:44, 12.58s/it]











86it [03:08,  9.03s/it]











87it [03:51, 19.25s/it]











117it [05:23, 14.39s/it]











138it [05:26, 10.12s/it]











142it [05:42,  8.24s/it]











149it [05:49,  6.07s/it]











152it [06:04,  5.79s/it]











160it [06:07,  4.15s/it]











165it [06:11,  3.16s/it]











168it [07:05,  7.59s/it]











184it [07:13,  5.46s/it]











195it [07:21,  4.06s/it]











213it [07:25,  2.89s/it]











215it [07:26,  2.20s/it]











217it [07:30,  2.12s/it]











226it [07:31,  1.52s/it]











230it [07:46,  2.23s/it]











236it [07:58,  2.17s/it]











243it [08:00,  1.60s/it]











246it [08:24,  3.49s/it]












ConnectionError: None: Max retries exceeded with url: /docs/downloads/1071/?f=%2Fuploadfiles%2Fprofstandarts%2F17.055.pdf (Caused by None)

In [73]:
_ = list(downloaded)













469it [23:33, 169.03s/it]

KeyboardInterrupt: 

TLDR: just adjusted `extract_fgosvo.py` to handle the part of profstandards that have got text layers, **SEE `extract_okz.py`** and `ps_to_okz.csv`. Out of 497 downloaded files, 349 had no easily extractable dataframes with OKZ codes.

Finally, **OKPDTR**

In [23]:
with requests.get('http://base.garant.ru/1548770/') as r:
    okpdtr_page = bs.BeautifulSoup(r.text)

In [24]:
okpdtr_page.find_all('pre')

[<pre class="s_2" id="p_42">     Х ХХХХ Х</pre>,
 <pre class="s_2" id="p_43">     |  |   \------контрольное число (КЧ)</pre>,
 <pre class="s_2" id="p_44">     |  |</pre>,
 <pre class="s_2" id="p_45">     |  \-----серийно-порядковый номер объекта классификации</pre>,
 <pre class="s_2" id="p_46">     |</pre>,
 <pre class="s_2" id="p_47">     |          профессия</pre>,
 <pre class="s_2" id="p_48">     \--------------------------------</pre>,
 <pre class="s_2" id="p_49">                должность</pre>,
 <pre class="s_2" id="p_92">-------------------------------------------------------------------------</pre>,
 <pre class="s_2" id="p_93">     |  |                                            |Диапазон|Код  |Код</pre>,
 <pre class="s_2" id="p_94"> Код |КЧ|           Наименование профессии           |тарифных|вы-  |по</pre>,
 <pre class="s_2" id="p_95">     |  |                                            |разрядов|пуска|ОКЗ</pre>,
 <pre class="s_2" id="p_96">     |  |                          

In [130]:
PAT_OKPDTR = re.compile(r'\b(\d{5})\s*(\d)\s+([^0-9]+)\s+[0-9].*$')
PAT_OKZ = re.compile(r'\b\d{4}\b')

def tokenize_pre(pre):
    m = PAT_OKPDTR.search(pre.text)
    if m:
        okpdtr = m.group(1) + m.group(2)
        yield ('okpdtr', okpdtr)
        yield ('namepart', m.group(3).strip())
    else:
        rest_of_name = (c for c in pre.contents if isinstance(c, str))    
        for namepart in rest_of_name:
            yield ('namepart', namepart)
    aa = pre.find_all('a')
    aa = (a.text for a in aa if a.text)
    aa = map(PAT_OKZ.search, aa)
    aa = filter(bool, aa)
    aa = (a.group(0) for a in aa)
    for a in aa:
        yield ('okz', a)
        
def compose_tokens(tokens):
    okpdtr, name, okz = None, [], None
    for event, data in tokens:
        if event not in ['okpdtr', 'namepart', 'okz']:
            raise Exception('Unknown event kind: {}'.format(event))
        if event == 'okpdtr':
            okpdtr = data
        if okpdtr is None:
            continue
        if event == 'namepart':
            name.append(data)
        elif event == 'okz':
            okz = data
            yield (okpdtr, okz, ' '.join(name))
            okpdtr, name, okz = None, [], None


with open('okz_to_okpdtr.csv', 'w') as f:
    out = csv.writer(f)
    out.writerow(['okpdtr', 'okz', 'name'])
    okpdtrs = okpdtr_page.find_all('pre')
    okpdtrs = map(tokenize_pre, okpdtrs)
    okpdtrs = itertools.chain.from_iterable(okpdtrs)
    okpdtrs = compose_tokens(okpdtrs)
    okpdtrs = tqdm.tqdm(okpdtrs)
    okpdtrs = map(tee(out.writerow), okpdtrs)
    okpdtrs = list(okpdtrs)




















0it [00:00, ?it/s]


















2320it [00:00, 23194.05it/s]


















4645it [00:00, 23209.69it/s]


















6882it [00:00, 22948.51it/s]


















7993it [00:00, 22723.50it/s]

In [131]:
len(okpdtrs)

7993

In [132]:
okpdtrs

[('100035', '7332', 'Авербандщик'),
 ('100054', '7515', 'Авиационный механик (техник) по планеру и'),
 ('100073', '7515', 'Авиационный механик (техник) по приборам и'),
 ('100088', '7515', 'Авиационный механик (техник) по радиообору-'),
 ('100105', '7515', 'Авиационный техник (механик)  по парашютным'),
 ('100124', '7515', 'Авиационный техник по горюче-смазочным'),
 ('100139', '8152', 'Автоклавщик'),
 ('100143', '8139', 'Автоклавщик'),
 ('100158', '8139', 'Автоклавщик'),
 ('100162', '8122', 'Автоклавщик литья под давлением'),
 ('100177', '8112', 'Автоклавщик на запарке брикетов'),
 ('100196', '8290', 'Автоклавщик-сушильщик аккумуляторных пластин'),
 ('100213', '8211', 'Автоматчик'),
 ('100232', '8224', 'Автоматчик вязальных автоматов'),
 ('100251', '7280', 'Автоматчик игольно-платинных изделий'),
 ('100270', '8143', 'Автоматчик картонажного производства'),
 ('100285', '8141', 'Автоматчик клеильных полуавтоматов'),
 ('100302', '8211', 'Автоматчик на узловязальных и навивочных'),
 ('1003